# 🏭 Smart Industrial Maintenance System — GPU Training Notebook

**FSE 570 Capstone** | Arizona State University

This notebook runs the complete training pipeline on Google Colab with GPU acceleration.

---

## 1. Setup Environment

In [ ]:
# Check GPU availability
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    props = torch.cuda.get_device_properties(0)
    vram = getattr(props, 'total_memory', getattr(props, 'total_mem', 0))
    print(f"VRAM: {vram / 1e9:.1f} GB")

In [ ]:
# Install dependencies
!pip install -q xgboost lifelines shap pulp

In [ ]:
# Clone your project repo
!git clone https://github.com/SivaKanth007/Capstone-Project.git
%cd Capstone-Project

## 2. Download & Preprocess Data

The download module will automatically try direct download first — **no Kaggle credentials needed**.
If the direct download fails, it falls back to a synthetic data generator.

In [ ]:
import os
import numpy as np
import config
from src.data.download import download_cmapss, load_cmapss_train
from src.data.preprocess import DataPreprocessor
from src.data.feature_engineering import FeatureEngineer
from src.data.synthetic_generator import SyntheticDataGenerator

# Download C-MAPSS dataset (direct URL, no auth)
download_cmapss()
df_train = load_cmapss_train()
print(f"\nTraining data: {df_train.shape}")
df_train.head()

In [ ]:
# Generate synthetic data
gen = SyntheticDataGenerator()
logs, context, schedule = gen.generate_all(df_train)
print(f"Maintenance logs: {logs.shape}")
print(f"Operational context: {context.shape}")

In [ ]:
# Feature engineering (for XGBoost)
fe = FeatureEngineer()
df_engineered = fe.engineer_features(df_train.copy())
print(f"Engineered features: {df_engineered.shape}")

In [ ]:
# Preprocessing pipeline (for LSTM models)
preprocessor = DataPreprocessor()
data = preprocessor.fit_transform(df_train)
preprocessor.save()

for split_name, split_data in data.items():
    np.savez_compressed(
        os.path.join(config.PROCESSED_DATA_DIR, f"{split_name}_data.npz"),
        **split_data
    )

X_train = data['train']['X']
y_train_rul = data['train']['y_rul']
y_train_binary = data['train']['y_binary']
X_val = data['val']['X']
y_val_binary = data['val']['y_binary']

n_features = X_train.shape[2]
print(f"Sequences: {X_train.shape}, Features: {n_features}")

## 3. Train LSTM Autoencoder (Anomaly Detection)

In [ ]:
from src.models.autoencoder import LSTMAutoencoder, AutoencoderTrainer

# Train on healthy data only
healthy_mask = y_train_rul > config.MAX_RUL * 0.5
X_healthy = X_train[healthy_mask]
X_val_ae = X_val[data['val']['y_rul'] > config.MAX_RUL * 0.5]

print(f"Training autoencoder on {len(X_healthy)} healthy samples")
print(f"Device: {config.DEVICE}")

autoencoder = LSTMAutoencoder(input_dim=n_features)
ae_trainer = AutoencoderTrainer(autoencoder, epochs=50)
ae_trainer.train(X_healthy, X_val_ae)
ae_trainer.save_model()

In [ ]:
# Visualize training loss
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(ae_trainer.train_history, label='Train Loss', color='#3a7bd5')
if ae_trainer.val_history:
    ax.plot(ae_trainer.val_history, label='Val Loss', color='#FF6B6B')
ax.set_xlabel('Epoch')
ax.set_ylabel('MSE Loss')
ax.set_title('Autoencoder Training')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Train LSTM Failure Predictor

In [ ]:
from src.models.lstm_predictor import LSTMPredictor, PredictorTrainer

predictor = LSTMPredictor(input_dim=n_features)
pred_trainer = PredictorTrainer(predictor, epochs=50)
pred_trainer.train(X_train, y_train_binary, X_val, y_val_binary)
pred_trainer.save_model()

In [ ]:
# Visualize predictor training
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))

ax1.plot(pred_trainer.train_history, label='Train Loss', color='#3a7bd5')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Predictor Training Loss')
ax1.legend()
ax1.grid(True, alpha=0.3)

if pred_trainer.val_history:
    epochs_list = range(5, len(pred_trainer.val_history) * 5 + 1, 5)
    f1s = [m.get('f1', 0) for m in pred_trainer.val_history]
    aucs = [m.get('auc', 0) for m in pred_trainer.val_history]
    ax2.plot(epochs_list, f1s, label='F1 Score', color='#44BB44', marker='o', markersize=4)
    ax2.plot(epochs_list, aucs, label='AUC', color='#FF6B6B', marker='s', markersize=4)
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Score')
    ax2.set_title('Validation Metrics')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Train XGBoost RUL Model

In [ ]:
from src.models.xgboost_rul import XGBoostRUL

exclude_cols = ['unit_id', 'cycle', 'RUL']
feature_cols = [c for c in df_engineered.columns if c not in exclude_cols]

unit_ids = df_engineered['unit_id'].unique()
np.random.seed(config.RANDOM_SEED)
np.random.shuffle(unit_ids)
n = len(unit_ids)
train_units = unit_ids[:int(n * 0.7)]
val_units = unit_ids[int(n * 0.7):int(n * 0.85)]

X_train_xgb = df_engineered[df_engineered['unit_id'].isin(train_units)][feature_cols]
y_train_xgb = df_engineered[df_engineered['unit_id'].isin(train_units)]['RUL'].values
X_val_xgb = df_engineered[df_engineered['unit_id'].isin(val_units)][feature_cols]
y_val_xgb = df_engineered[df_engineered['unit_id'].isin(val_units)]['RUL'].values

xgb_model = XGBoostRUL()
xgb_model.train(X_train_xgb, y_train_xgb, X_val_xgb, y_val_xgb,
                feature_names=feature_cols)
xgb_model.evaluate(X_val_xgb, y_val_xgb)
xgb_model.save()

## 6. Bayesian Survival Analysis

In [ ]:
from src.models.bayesian_survival import BayesianSurvival

survival_features = config.ACTIVE_SENSORS + ['cycle']
survival_cols = [c for c in survival_features if c in df_train.columns] + ['RUL']

df_survival_train = df_train[df_train['unit_id'].isin(train_units)][['unit_id'] + survival_cols]

survival_model = BayesianSurvival()
survival_model.fit(df_survival_train)

df_survival_val = df_train[df_train['unit_id'].isin(val_units)][['unit_id'] + survival_cols]
survival_model.evaluate(df_survival_val)
survival_model.save()

## 7. Explainability (SHAP & Attention)

In [ ]:
from src.explainability.shap_analysis import SHAPExplainer

# SHAP for XGBoost
shap_explainer = SHAPExplainer(xgb_model, model_type='xgboost')
shap_explainer.compute_shap_values(X_val_xgb)
shap_explainer.plot_global_importance(save_path='shap_importance.png')
shap_explainer.plot_beeswarm(save_path='shap_beeswarm.png')
ranking = shap_explainer.get_sensor_ranking()
ranking.head(15)

In [ ]:
from src.explainability.attention_viz import AttentionVisualizer

# Attention visualization
attn_viz = AttentionVisualizer(predictor)
attn_viz.plot_attention_heatmap(data['test']['X'], save_path='attention_heatmap.png')
attn_viz.plot_average_attention(data['test']['X'], data['test']['y_binary'],
                                save_path='attention_comparison.png')

## 8. MILP Optimization & Simulation

In [ ]:
from src.optimization.milp_scheduler import MaintenanceScheduler

# Get predictions for test data
failure_proba, _ = predictor.predict_proba(torch.FloatTensor(data['test']['X']).to(config.DEVICE))

# Aggregate per unit
unit_risks = {}
for uid in np.unique(data['test']['unit_ids']):
    mask = data['test']['unit_ids'] == uid
    unit_risks[int(uid)] = float(failure_proba[mask][-1])

# Run MILP optimization
scheduler = MaintenanceScheduler()
result = scheduler.create_schedule(
    machine_risks=unit_risks,
    machine_names={uid: f'Engine-{uid:03d}' for uid in unit_risks}
)
result['schedule']

In [ ]:
# Monte Carlo simulation
from src.evaluation.simulation import MaintenanceSimulator

sim = MaintenanceSimulator(n_machines=50, n_periods=100)
sim_df, sim_summary = sim.run_comparison(n_simulations=100)
sim.plot_comparison(sim_df, save_path='simulation_comparison.png')

## 9. Download Results

Download trained models and results back to local machine.

In [ ]:
# Save all results to a zip for download
import shutil
shutil.make_archive('capstone_results', 'zip', '.', 'models/saved')

# In Colab, download the zip:
try:
    from google.colab import files
    files.download('capstone_results.zip')
except ImportError:
    print('Not in Colab. Files saved locally.')

---
**Project**: FSE 570 Data Science Capstone | Arizona State University